In [1]:
import requests

import pandas as pd

from bs4 import BeautifulSoup

import time

pd.set_option('display.max_columns', 50)

In [2]:
years = list(range(2024,2021, -1))
years

[2024, 2023, 2022]

In [3]:
connect_timeout = 6
read_timeout = 100

def stats(link, year):
     
    data = requests.get(link, timeout=(connect_timeout, read_timeout))
    soup = BeautifulSoup(data.text)
    
    stats_table = pd.read_html(data.text, match="Regular season Table")[0]
    
    standard_stats = pd.read_html(data.text, match = "Squad Standard Stats")[0]
    standard_stats.columns = standard_stats.columns.droplevel()
    
    goalkeeping_stats = pd.read_html(data.text, match="Squad Goalkeeping")[0]
    goalkeeping_stats.columns = goalkeeping_stats.columns.droplevel()
    
    shooting_stats = pd.read_html(data.text, match="Squad Shooting")[0]
    shooting_stats.columns = shooting_stats.columns.droplevel()
    
    stats_standard = stats_table.merge(standard_stats[["Squad", "Poss", "Gls", "Ast"]], on="Squad", how="left")
    stats_standard_gk = stats_standard.merge(goalkeeping_stats[["Squad", "CS%"]], on="Squad", how="left")
    all_stats = stats_standard_gk.merge(shooting_stats[["Squad", "SoT/90", "G/SoT", "SoT%"]], on="Squad", how="left")
        
    all_stats["Season"] = year
    
    return all_stats
    

In [4]:
overall_stats = []

In [5]:
def concate():
    
    all_stats_2024 = stats("https://fbref.com/en/comps/9/Premier-League-Stats", 2024)
    all_stats_2024 = all_stats_2024.drop(["Last 5"], axis=1)
    all_stats_2023 = stats("https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats", 2023)
    all_stats_2022 = stats("https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats", 2022)
    
    
    overall_stats.append(all_stats_2024)
    overall_stats.append(all_stats_2023)
    overall_stats.append(all_stats_2022)
    
    overall_stats_df = pd.concat(overall_stats)
    return overall_stats_df
    

In [6]:
overall_stats_df = concate()

In [9]:
overall_stats_df.head()

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Pts/MP,xG,xGA,xGD,xGD/90,Attendance,Top Team Scorer,Goalkeeper,Notes,Poss,Gls,Gls,Ast,Ast,CS%,SoT/90,G/SoT,SoT%,Season
0,1,Liverpool,20,13,6,1,43,18,25,45,2.25,44.1,22.6,21.5,1.08,46872,Mohamed Salah - 14,Alisson,NaN,60.2,40,2.00,30,1.50,35.0,6.15,0.29,32.9,2024
1,2,Aston Villa,20,13,3,4,43,27,16,42,2.10,36.0,25.6,10.3,0.52,28969,Ollie Watkins - 9,Emiliano Martínez,NaN,54.6,40,2.00,30,1.50,20.0,5.10,0.35,35.3,2024
2,3,Manchester City,19,12,4,3,45,21,24,40,2.11,36.8,17.8,18.9,1.00,47396,Erling Haaland - 14,Ederson,NaN,63.6,44,2.32,32,1.68,26.3,6.26,0.34,38.9,2024
3,4,Arsenal,20,12,4,4,37,20,17,40,2.00,36.4,16.3,20.0,1.00,60169,Bukayo Saka - 6,David Raya,NaN,60.3,36,1.80,23,1.15,35.0,4.90,0.31,30.7,2024
4,5,Tottenham,20,12,3,5,42,29,13,39,1.95,35.2,34.7,0.4,0.02,61624,Son Heung-min - 12,Guglielmo Vicario,NaN,59.2,39,1.95,34,1.70,25.0,5.85,0.32,36.7,2024


In [10]:
overall_stats_df.to_csv("overall_stats.csv", index=False)